## Baseline Scenario

#### Fixed Split:

- Bandwidth: 50% to user equipment, 50% to sensors
- Computational power: 50% to user equipment, 50% to sensors

In [ ]:
import gymnasium as gym
import matplotlib.pyplot as plt

# predefined smart city scenario
from mobile_env.scenarios.smart_city import MComSmartCity

In [ ]:
env = gym.make("mobile-smart_city-smart_city_handler-v0", render_mode="rgb_array")

print(f"\nSmart city environment with {env.NUM_USERS} users, {env.NUM_SENSORS} sensors and {env.NUM_STATIONS} cells.")

In [ ]:
from IPython import display

done = False
obs, info = env.reset()

for _ in range(100):
    dummy_action = (0.50, 0.50)
    obs, reward, terminated, truncated, info = env.step(dummy_action)

    plt.imshow(env.render())
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [ ]:
scalar_results, kpi_results, ue_results, bs_results, ss_results = env.monitor.load_results()

scalar_results

In [ ]:
kpi_results

In [ ]:
ue_results

In [ ]:
ss_results

In [ ]:
bs_results

In [ ]:
data = kpi_results.agg(['min', 'max', 'mean'])
data

In [ ]:
data = scalar_results.agg(['min', 'max', 'mean'])
data

In [ ]:
data = ue_results.reset_index()
data = data.groupby('UE ID').agg(['mean', 'min', 'max'])
data = data.drop('Time Step', axis='columns')
data

In [ ]:
data = ss_results.reset_index()
data = data.groupby('Sensor ID').agg(['mean', 'min', 'max'])
data = data.drop('Time Step', axis='columns')
data

In [ ]:
data = bs_results.reset_index()
data = data.groupby('BS ID').agg(['mean', 'min', 'max'])
data = data.drop('Time Step', axis='columns')
data

In [ ]:
import matplotlib.pyplot as plt

def plot_kpi_metrics():
    # Plot kpi metrics
    for metric in kpi_results.columns:
        plt.figure(figsize=(12, 8))
        kpi_results[metric].plot()
        plt.title(f"{metric}")
        plt.xlabel("Time Step")
        plt.ylabel("Value")
        plt.grid(True)
        plt.show()

plot_kpi_metrics()

In [ ]:
import pandas as pd
import time

# Reset environment and initialize variables
done = False
obs, info = env.reset()
all_kpi_means = []  # List to store mean metrics for each episode
episode_infos = []

# Run for 100 time steps for 100 episodes
for episode in range(100):
    obs, info = env.reset()
    print(f"Starting Episode {episode + 1}")
    episode_start_time = time.time()  # Start timing the episode

    for t in range(100):
        # Perform dummy action
        dummy_action = (0.50, 0.50)
        obs, reward, terminated, truncated, info = env.step(dummy_action)

        # Check termination condition
        if terminated or truncated:
            break
    
    # Calculate and store the episode duration
    episode_duration = time.time() - episode_start_time
    print(f"Completed Episode {episode + 1}: {episode_duration}")

    # Load results after the episode
    _, kpi_results, _, _, _ = env.monitor.load_results()

    # Compute mean of metrics over the episode
    kpi_mean = kpi_results.mean()  # Calculate mean for each metric
    all_kpi_means.append(kpi_mean)

    # Add episode-level information to the list
    episode_info = {
        "Episode": episode + 1,
        "Total Reward": info.get("episode reward"),
        "Queue Size Transferred Jobs UE": info.get("bs trans. ue"),
        "Queue Size Transferred Jobs Sensor": info.get("bs trans. ss"),
        "Queue Size Accomplished Jobs UE": info.get("bs accomp. us"),
        "Queue Size Accomplished Jobs Sensor": info.get("bs accomp. ss"),
    }
    episode_infos.append(episode_info)

    print(f"Mean Reward for Episode: {episode_info['Total Reward']/100:.2f}")


# Convert all_kpi_means and episode_infos to DataFrames
episode_infos_df = pd.DataFrame(episode_infos)
all_kpi_means_df = pd.DataFrame(all_kpi_means)

# Display mean KPI metrics for each episode
print(episode_infos_df)
print(all_kpi_means_df)

# Save results to CSV files
episode_infos_df.to_csv("episode_infos.csv", index=False)
all_kpi_means_df.to_csv("kpi_means_per_episode.csv", index=False)